In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [2]:
neg_deceptive_folder_path =r'C:\Users\KIIT\Downloads\op_spam_v1.4\op_spam_v1.4\negative_polarity\deceptive_from_MTurk' 
neg_true_folder_path = r'C:\Users\KIIT\Downloads\op_spam_v1.4\op_spam_v1.4\negative_polarity\truthful_from_Web'
pos_deceptive_folder_path =r'C:\Users\KIIT\Downloads\op_spam_v1.4\op_spam_v1.4\positive_polarity\deceptive_from_MTurk'
pos_true_folder_path = r'C:\Users\KIIT\Downloads\op_spam_v1.4\op_spam_v1.4\positive_polarity\truthful_from_TripAdvisor'

In [3]:
polarity_class = []
reviews = []
spamity_class =[]

In [4]:
for i in range(1,6):
    insideptru = pos_true_folder_path + '\\fold' + str(i) 
    insidepdec = pos_deceptive_folder_path + '\\fold' + str(i)
    insidentru = neg_true_folder_path + '\\fold' + str(i) 
    insidendec = neg_deceptive_folder_path + '\\fold' + str(i) 
    pos_list = []
    for data_file in sorted(os.listdir(insidendec)):
        polarity_class.append('negtive')
        spamity_class.append(str(data_file.split('_')[0]))
        with open(os.path.join(insidendec, data_file)) as f:
                contents = f.read()
                reviews.append(contents)
    for data_file in sorted(os.listdir(insidentru)):
        polarity_class.append('negative')
        spamity_class.append(str(data_file.split('_')[0]))
        with open(os.path.join(insidentru, data_file)) as f:
                contents = f.read()
                reviews.append(contents)
    for data_file in sorted(os.listdir(insidepdec)):
        polarity_class.append('positive')
        spamity_class.append(str(data_file.split('_')[0]))
        with open(os.path.join(insidepdec, data_file)) as f:
                contents = f.read()
                reviews.append(contents)
    for data_file in sorted(os.listdir(insideptru)):
        polarity_class.append('positive')
        spamity_class.append(str(data_file.split('_')[0]))
        with open(os.path.join(insideptru, data_file)) as f:
                contents = f.read()
                reviews.append(contents)

In [5]:
data_fm = pd.DataFrame({'polarity_class':polarity_class,'review':reviews,'spamity_class':spamity_class})

data_fm.loc[data_fm['spamity_class']=='d','spamity_class']=0
data_fm.loc[data_fm['spamity_class']=='t','spamity_class']=1

In [6]:
data_x = data_fm['review']

data_y = np.asarray(data_fm['spamity_class'],dtype=int)

#X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

In [7]:
data_fm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 3 columns):
polarity_class    1600 non-null object
review            1600 non-null object
spamity_class     1600 non-null object
dtypes: object(3)
memory usage: 37.6+ KB


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
data_x = vectorizer.fit_transform(data_x)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

In [10]:
#cv =  CountVectorizer()
#X_traincv = cv.fit_transform(X_train)
#X_testcv = cv.transform(X_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier as rfc

In [12]:
classifier = rfc()
classifier.fit(X_train,y_train)
classifier.score(X_test,y_test)

C:\Users\KIIT\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.725

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

classifier = rfc()

hyperparameters = {
    'n_estimators':stats.randint(10,300),
    'criterion':['gini','entropy'],
    'min_samples_split':stats.randint(2,9),
    'bootstrap':[True,False]
}

random_search = RandomizedSearchCV(classifier, hyperparameters, n_iter=65, n_jobs=4)

random_search.fit(data_x,data_y)

C:\Users\KIIT\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=65, n_jobs=4,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F97CC4EEF0>, 'criterion': ['gini', 'entropy'], 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F97CDE20F0>, 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [14]:
optimized_classifier = random_search.best_estimator_
optimized_classifier.fit(X_train,y_train)

optimized_classifier.score(X_test,y_test)

0.86875

In [15]:
optimized_classifier.fit(data_x,data_y)

sentence = vectorizer.transform(['Every time I opened this app with in few minutes it has\
reloaded itself and mobile go down slowly why this is happening in only to this telegram app\
only all other apps running too fast but telegram will not work in my mobile why\
( it work excellent in past few months , but for the last two months it works slowly)'])


optimized_classifier.predict_proba(sentence)

array([[0.37027778, 0.62972222]])